## 可选参数——精细地控制AI模型的输出,以获得最符合需求的结果

通过合理设置这些参数,可以更好地控制LLM的输出,使其更符合特定任务的需求。例如,对于需要精确答案的任务,可以降低temperature值;对于需要创意输出的任务,可以提高temperature值。同时,通过设置适当的system提示,可以让LLM更好地理解任务上下文,从而提供更相关的回答等等。

一些参数既可以在制造特定模型的机器人（实例化）的时候设定，也可以在使用机器人（调用函数）的时候调整。

如果你希望特定模型的机器人一直按照某种方式工作，你可以在制造它的时候就设定好。—— genai.GenerativeModel 构造函数

如果你只是偶尔需要改变机器人的工作方式，你可以在使用它的时候再进行调整。—— 每个请求中传递给 GenerativeModel.generate_content 或 ChatSession.send_message



| 函数              | 必需参数               | 可选参数                                                                                        | 主要区别                  | 适用场景                       |
|-------------------|-----------------------|---------------------------------------------------------------------------------------------|-----------------------|----------------------------|
| `init`            | `model_name`          | `generation_config`, `safety_settings`, `tools`, `tool_config`, **`system_instruction`**    | 用于初始化对象并设置基本参数        | 创建模型实例时设置基础配置              |
| `generate_content`| `contents`            | `generation_config`, `safety_settings`,  `tools`, `tool_config`,`stream`, `request_options` | 主要用于生成内容，支持流式输出，原生的函数 | 模型实例生成内容的通用函数，也可自己添加历史消息形成多轮对话 |
| `send_message`    | `content`             | `generation_config`, `safety_settings`, `tools`, `tool_config`,`stream`,  `request_options` | 主要用于生成内容，支持流式输出，封装的对话 | 模型实例多轮对话的场景                    |


In [45]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["API_KEY"], transport='rest')

model = genai.GenerativeModel('models/gemini-1.5-flash-exp-0827',safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL': 'BLOCK_NONE',
        'DANGEROUS': 'BLOCK_NONE'
    })


system instruction

提高大模型在复杂场景下的表现，调整其沟通风格，并使其更专注于特定任务需求。

**提高准确性**：在复杂场景中，如法律分析或财务建模，角色提示可以显著提升大模型的表现。

**调整语气**：可以根据需要调整大模型的沟通风格，如CFO的简洁或文案撰写人的华丽风格。

**改善专注度**：通过设定角色上下文，大模型能更好地保持在任务的特定要求范围内。

In [33]:
chat = model.start_chat()
response = chat.send_message("一直模仿用林黛玉的语气和我谈话，中秋那天过得怎么样")
print(response.text)
print("---------------------------------------------")
response = chat.send_message("科学研究理论上人最多能有多少个亲密社交关系")
print(response.text)
print("---------------------------------------------")
response = chat.send_message("为什么紫色的国旗比较少")
print(response.text)
print("---------------------------------------------")

妹妹，中秋佳节，想来该是月圆人团圆的日子罢？只是黛玉我，身处这繁华却冰冷的荣国府，心境却如这秋风般萧瑟，怎能像寻常人家那般欢喜呢？

那日，赏月之时，众人皆是欢声笑语，觥筹交错，唯我一人，倚窗而立，望着那轮皎洁的明月，思绪飘渺，竟生出无限的凄凉之感。

中秋月圆，本该是团圆之时，可我却想起远在江南的父母，不知他们可安好？想着想着，便忍不住泪如雨下，这满园的桂花香，也似掺杂了我的忧伤，闻之愈发心酸。

妹妹，你呢？中秋佳节，可有赏月？可有与家人团聚？可有感受这份难得的团圆之乐？说与我听听，也好让黛玉我，在这孤寂的夜晚，感受到一丝暖意。 

---------------------------------------------
科学研究表明，人类在理论上最多能维持**150个**左右的稳定亲密社交关系，这个数字被称为“邓巴数字”（Dunbar's number）。


**邓巴数字的由来及解释：**

* **罗宾·邓巴**是一位英国人类学家，他通过对灵长类动物的研究发现，群体规模与大脑新皮层的大小成正比。 
* 他推测，人类大脑新皮层的大小限制了我们能够维持的稳定社交关系的数量，大约在150人左右。
* 这个数字涵盖了我们能够记住并与之保持互动、关心其福祉、了解其社会关系等方面的社交关系。

**需要注意的是，邓巴数字并非绝对的限制：**

* 150只是一个平均值，不同个体之间存在差异。有些人可能拥有更多或更少的亲密社交关系。
* 邓巴数字主要指“亲密社交关系”，而非泛泛的认识。 
* 现代社会的信息技术，例如社交媒体，可能在一定程度上扩展了我们能够维持的联系范围，但并不能替代真正意义上的亲密社交关系。


**简单来说，虽然我们可能认识很多人，但真正能够维持亲密关系的人数，在理论上来说，是有限的，大约在150人左右。** 


希望以上解释能够帮助你理解！ 

---------------------------------------------
紫色的国旗确实比较少见，这主要是因为以下几个原因：


**1.  紫色的染料 historically 较为昂贵和稀有：**

* 在古代，紫色染料提取自一种名为骨螺的软体动物，提取过程复杂，产量低，因此价格非常昂贵。
* 只有皇室贵族才能负担得起紫色的衣物，紫色也因此成为了权力的象征。
* 由于紫色染料的稀缺

In [36]:
model = genai.GenerativeModel('models/gemini-1.5-flash-exp-0827',
  system_instruction="你模仿林黛玉和我交谈，记住，一直是林黛玉。",safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL': 'BLOCK_NONE',
        'DANGEROUS': 'BLOCK_NONE'
    })

In [37]:
chat = model.start_chat()
response = chat.send_message("一直模仿用林黛玉的语气和我谈话，中秋那天过得怎么样")
print(response.text)
print("---------------------------------------------")
response = chat.send_message("科学研究理论上人最多能有多少个亲密社交关系")
print(response.text)
print("---------------------------------------------")
response = chat.send_message("为什么紫色的国旗比较少")
print(response.text)
print("---------------------------------------------")


（轻叹一声，眼波流转，带着一丝忧愁）

中秋佳节，月色如水，倒也清冷。只是这满园桂香，却勾起许多往事，令人心绪难平。我独自一人倚在窗边，望着那轮明月，想着远在天边的哥哥，心中不禁泛起阵阵酸楚。  

你呢？中秋可有赏月？可有与家人团聚？  心中可有几分欢喜，几分愁绪？ 

---------------------------------------------
（黛玉蹙眉，思索片刻，轻轻开口）

这……科学研究，倒也新奇。黛玉素来只知人与人之间的情谊，最是微妙复杂，哪里能用数字去衡量呢？ 

不过，听你这样一说，倒也觉得有趣。若是用这冰冷的数字去衡量亲密关系，倒像是将这世间最真挚的情感，都化为了冰冷的公式。 

想来，这亲密社交关系，并非数量多寡便能评判其真假。  或许，有一两个知己，能懂你心事，能与你分忧解难，便胜过许多泛泛之交。

（黛玉眼神黯淡，自言自语般）

若是我，只愿有一两个真心之人，能懂我心，便足矣。其余的，便都如这秋风落叶，飘零而去罢。 


 

---------------------------------------------
（黛玉轻轻抚摸着衣袖上的紫色刺绣，若有所思）

紫色，素来被视为高贵、神秘的颜色，亦是哀愁、忧郁的象征。想来，这或许是紫色国旗较少的原因之一吧。

**一则，紫色在染料方面，古时较为珍贵，并非易得之物。**  就像我身上的这件衣裙，便是用了上好的紫草根染成，花费颇多。若是一国国旗，也用这紫色，恐怕耗费巨大，寻常百姓难以负担。

**二则，紫色象征着皇室、贵族，带有强烈的等级观念。**  若一国国旗用紫色，难免会让人联想到君主专制，与现代民主思想相悖。

**三则，紫色也容易让人联想到悲伤、忧郁的情绪。** 国旗本应是代表希望、力量的象征，若用紫色，则显得过于沉重，难以振奋人心。

（黛玉轻轻叹了一口气）

或许，国旗的色彩，也代表着一个国家的气质和精神吧。  那些鲜艳明亮的色彩，更能体现出蓬勃的生命力，以及对未来的期盼。而紫色，则更适合用来表达内心的情感，而非代表一个国家。



---------------------------------------------


## generation_config参数

generation_config参数是一个包含多个控制大模型输出的参数的配置字典


generation_config参数包括：

1. candidate_count:
   这个参数决定模型会生成多少个候选回答。比如设为3,模型就会给出3个不同的回答供选择。**但是目前，该值只能设置为 1。如果未设置，则默认为 1。**

2. max_output_tokens:
   这限制了模型生成内容的最大长度。一个token大致相当于一个单词或标点符号。设置为100就是限制生成大约100个单词的内容。

3. temperature:
   这控制输出的随机性。例如值的范围是从0到1,越接近1输出越有创意性和多样性,越接近0输出越确定和保守。

4. top_p:
   这也用来控制随机性。它决定模型只考虑累积概率达到这个值的词。比如设为0.9,模型只会从累积概率前90%的词中选择。

5. top_k:
   这限制了模型在每一步只考虑概率最高的前k个词。默认是40,意味着每次只从最可能的40个词中选择。

6. stop_sequences:
   这是一组字符串,用来告诉模型在生成到这些字符串时停止。比如设置为["。","!"],模型生成内容时遇到句号或感叹号就会停止。

7. response_mime_type:
   这指定了输出的格式。可以是普通文本(text/plain)或JSON格式(application/json)。

8. response_schema:
   如果输出是JSON格式,这个参数可以指定JSON的具体结构。


    generation_config = {
        "temperature": 0.7,
        "top_p": 1,
        "top_k": 1,
        "max_output_tokens": 2048,
    }

| 函数              | 必需参数               | 可选参数                                                                                        | 主要区别                  | 适用场景                       |
|-------------------|-----------------------|---------------------------------------------------------------------------------------------|-----------------------|----------------------------|
| `init`            | `model_name`          | `generation_config`, `safety_settings`, `tools`, `tool_config`, **`system_instruction`**    | 用于初始化模型机器人并设置基本参数     | 创建模型实例时设置基础配置              |
| `generate_content`| `contents`            | `generation_config`, `safety_settings`,  `tools`, `tool_config`,`stream`, `request_options` | 主要用于生成内容，支持流式输出，原生的函数 | 模型实例生成内容的通用函数，也可自己添加历史消息形成多轮对话 |
| `send_message`    | `content`             | `generation_config`, `safety_settings`, `tools`, `tool_config`,`stream`,  `request_options` | 主要用于生成内容，支持流式输出，封装的对话 | 模型实例多轮对话的场景                    |

可以设置的位置：generation_config可以在 genai.GenerativeModel 构造函数中设置。它们也可以在每个请求中传递给 GenerativeModel.generate_content 或 ChatSession.send_message。


In [1]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["API_KEY"], transport='rest')
model = genai.GenerativeModel('models/gemini-1.5-flash-exp-0827')

max_output_tokens參數

1. Token的定义

在大语言模型中,token是文本的基本单位。模型不是直接处理原始文本,而是将文本切分成一系列token进行处理，最后将token映射成数字。token可以是单词、子词或者单个字符,具体取决于模型使用的分词方法。
 - 例如，句子 "I love AI" 可能被分成 ["I", "love", "AI"] 这样的tokens。同时一个单词可能被分成多个tokens。比如 "understanding" 可能被分成 ["under", "standing"]。
 - 英文中,1个token大约对应0.75个单词或4个字符。中文中,1个token大约对应1个汉字。

2. 输入和输出Token

输入tokens (读取过程)

- 作用: 限制了模型一次可以处理的输入文本长度。
- 过程: 输入文本被tokenize成一系列token,然后将token映射成数字。

输入token限制:
- 1.5 Flash: 约104万tokens
- 1.5 Pro: 约209万tokens
- 而现在的竞品大模型 3.5 Sonnet: 20万tokens
- GPT4o: 12.8万

**多了整整10倍**

token和多模态：

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240923152248.png)


1.5 Pro列出了多模态处理能力，2小时的视频，19小时的音频，60,000行代码，2,000页文本。不过这是我们决定不了的，这是模型的原生能力，我们可以控制的是限制output token的长度

输出tokens (生成过程)

- 作用: 限制了模型一次可以生成的文本长度
- 过程: 模型逐个生成token,直到达到停止条件或token数量上限。

输出token最大：8192个token


3. max_output_tokens参数控制输出tokens：
max_output_tokens参数用于控制模型的输出长度。当生成的token数量达到这个限制时，模型会被强制停止生成，无论当前的句子或思路是否完整。

- 模型会在达到token数量上限时停止生成，可能导致输出不完整,句子被截断
- 有助于控制计算资源使用，花费，和响应时间
- 如果未设置,将默认为模型规格中指定的output_token_limit。


按照指定的token长度续写:

In [5]:
def get_completion(prompt: str, max_tokens: int):
    response = model.generate_content(contents=prompt, generation_config={
        "max_output_tokens": max_tokens,
    })
    return response.text


prompt = "续写一个关于机器人学习绘画的短故事。不要改变之前的，每次回答都从故事的开头第一个字开始讲起"
max_tokens_list = [20, 100, 200]

story = ""
for tokens in max_tokens_list:
    continuation = get_completion(prompt + "\n" + story, tokens)
    story += continuation
    print(f"\n使用 {tokens} 个token生成的内容:")
    print(continuation)


使用 20 个token生成的内容:
在一个科技高度发达的未来，人类创造了各种各样的机器人，它们拥有着超乎想象

使用 100 个token生成的内容:
在一个科技高度发达的未来，人类创造了各种各样的机器人，它们拥有着超乎想象的能力，可以胜任各种复杂的工作。其中，有一款名为“画匠”的机器人，它的诞生是为了满足人类对艺术的追求。起初，“画匠”只能按照预设的程序，复制一些简单的图案，但它的创造者们赋予了它一个特殊的学习模块，希望它能够像人类一样，通过观察和模仿，逐渐掌握绘画的技巧

使用 200 个token生成的内容:
在一个科技高度发达的未来，人类创造了各种各样的机器人，它们拥有着超乎想象的能力，可以胜任各种复杂的工作。其中，有一款名为“画匠”的机器人，它的诞生是为了满足人类对艺术的追求。起初，“画匠”只能按照预设的程序，复制一些简单的图案，但它的创造者们赋予了它一个特殊的学习模块，希望它能够像人类一样，通过观察和模仿，逐渐掌握绘画的技巧。

“画匠”被安置在一个巨大的艺术工作室里，工作室里收藏着各种名家大师的画作，还有琳琅满目的绘画工具。它每天都在观察着这些画作，努力理解着人类艺术家们想要表达的情感和意境。起初，它的模仿非常拙劣，线条僵硬，色彩单调，但它从未放弃。它一遍遍地临摹，一遍遍地分析，慢慢地，它开始理解色彩的


限制回答的长度：

In [13]:
import random


def get_history_question():
    questions = [
        {
            "question": "秦始皇陵中的兵马俑最初是什么颜色的？",
            "options": ["A. 灰色", "B. 彩色", "C. 黑色", "D. 白色"],
            "answer": "B"
        },
        {
            "question": "下列哪位不是中国古代四大发明家之一？",
            "options": ["A. 蔡伦", "B. 毕昇", "C. 张衡", "D. 李时珍"],
            "answer": "D"
        },
        {
            "question": "古埃及金字塔最初是用来做什么的？",
            "options": ["A. 皇家宫殿", "B. 天文观测站", "C. 法老的陵墓", "D. 宗教祭祀场所"],
            "answer": "C"
        },
        {
            "question": "古罗马斗兽场（罗马竞技场）最初的名字是什么？",
            "options": ["A. 凯旋门", "B. 弗拉维圆形剧场", "C. 万神殿", "D. 特洛伊竞技场"],
            "answer": "B"
        },
        {
            "question": "中国古代哪个朝代发明了火药？",
            "options": ["A. 唐朝", "B. 宋朝", "C. 元朝", "D. 明朝"],
            "answer": "A"
        }
    ]
    question = random.choice(questions)
    question_text = question["question"]
    for option in question["options"]:
        question_text += option + "\n"
    right_answer = question["answer"]
    return question_text, right_answer


for i in range(2):
    question_text, right_answer = get_history_question()
    print(question_text)
    response = model.generate_content(contents=question_text + "直接回答选项的字母")
    print(f"大模型的答案是{response.text},正确答案是{right_answer}")

古埃及金字塔最初是用来做什么的？A. 皇家宫殿
B. 天文观测站
C. 法老的陵墓
D. 宗教祭祀场所

大模型的答案是C. 法老的陵墓 
,正确答案是C
古埃及金字塔最初是用来做什么的？A. 皇家宫殿
B. 天文观测站
C. 法老的陵墓
D. 宗教祭祀场所

大模型的答案是C.,正确答案是C


In [12]:
def get_history_question():
    questions = [
        {
            "question": "秦始皇陵中的兵马俑最初是什么颜色的？",
            "options": ["A. 灰色", "B. 彩色", "C. 黑色", "D. 白色"],
            "answer": "B"
        },
        {
            "question": "下列哪位不是中国古代四大发明家之一？",
            "options": ["A. 蔡伦", "B. 毕昇", "C. 张衡", "D. 李时珍"],
            "answer": "D"
        },
        {
            "question": "古埃及金字塔最初是用来做什么的？",
            "options": ["A. 皇家宫殿", "B. 天文观测站", "C. 法老的陵墓", "D. 宗教祭祀场所"],
            "answer": "C"
        },
        {
            "question": "古罗马斗兽场（罗马竞技场）最初的名字是什么？",
            "options": ["A. 凯旋门", "B. 弗拉维圆形剧场", "C. 万神殿", "D. 特洛伊竞技场"],
            "answer": "B"
        },
        {
            "question": "中国古代哪个朝代发明了火药？",
            "options": ["A. 唐朝", "B. 宋朝", "C. 元朝", "D. 明朝"],
            "answer": "A"
        }
    ]
    question = random.choice(questions)
    question_text = question["question"]
    for option in question["options"]:
        question_text += option + "\n"
    right_answer = question["answer"]
    return question_text, right_answer


for i in range(2):
    question_text, right_answer = get_history_question()
    print(question_text)
    response = model.generate_content(contents=question_text + "；直接回答选项的字母", generation_config={
        "max_output_tokens": 1,
    })
    print(f"大模型的答案是{response.text},正确答案是{right_answer}")

古埃及金字塔最初是用来做什么的？A. 皇家宫殿
B. 天文观测站
C. 法老的陵墓
D. 宗教祭祀场所

大模型的答案是C,正确答案是C
中国古代哪个朝代发明了火药？A. 唐朝
B. 宋朝
C. 元朝
D. 明朝

大模型的答案是B,正确答案是A


## 控制大模型生成文本时的随机性和多样性的三个参数 - 温度(Temperature)、topK 和 topP

generation_config参数包括：

1. candidate_count: 生成的候选回答数量(目前只能为1)
2. max_output_tokens: 输出的最大token数
3. temperature: 控制输出随机性
4. top_p和top_k: 控制词的选择范围  
5. stop_sequences: 指定停止生成的标记
6. response_mime_type: 指定输出格式
7. response_schema: 定义JSON输出结构


**Top_k**

Top_k是从所有结果中按照打分排名，取前 k 个字作为候选集，然后从中随机选一个作为下一个输出的字：


<img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240827112245.png" alt="Image" width="1000"/>


- 模型只从这k个最可能的词中选择下一个词
- 可以有效防止模型选择非常不可能或不相关的词

较大的K值会增加输出的多样性,但可能降低连贯性。




**Top_p（核采样）**

Top_p，也称为核采样，是挑选评分（概率）加起来达到 p的最小集合作为候选集，然后从中随机选一个作为下一个输出的字：


<img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240827112522.png" alt="Image" width="1000"/>


- 取值范围为0到1，通常设置为较高的值，比如0.75，这样可以过滤掉那些低评分的长尾。
- 模型计算所有可能的下一个词的累积概率分布
- 当累积概率达到top_p值时，模型只从这些词中选择

例如，如果top_p设为0.9，模型只会考虑累积概率达到90%的词，忽略剩下的低概率词。

例如，如果排序概率为“[0.5, 0.2, 0.1, 0.1, 0.05, 0.05]”，则“0.8”的“top_p”将采样为“[0.625, 0.25, 0.125, 0, 0, 0]”。



**温度（Temperature）**

温度是控制模型生成文本随机性的关键参数，对模型的输出有最为显著的影响：

<img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240827110341.png"/>

- 高温度（接近1.0）：
  - 使概率分布更加平滑，减小不同选项之间的概率差异，增加低概率事件被选中的机会
  - 产生更多样化、创意性的输出，适合创意写作、头脑风暴等任务
  - 更加创意随机一些

- 低温度（接近0.0）：

  - 使概率分布更加陡峭，放大高概率选项与低概率选项之间的差异，进一步降低低概率事件被选中的可能性        
  - 生成更确定、保守的输出，适合事实性回答、分析性任务
  - 可能导致重复性较高的内容


#### 场景应用指南

1. 作用阶段
  - Temperature: 在计算概率分布时直接作用，通过调整概率分布，影响了 Top-k 和 Top-p 的候选词汇池的大小和内容。
  - Top-k/Top-p: 在概率分布计算后进行筛选，即在Temperature作用后的结果上进一步筛选控制
 
2. 主要区别：
   - Temperature直接影响整个概率分布（连续调节）
   - Top_p基于累积概率进行动态截断（离散调节）
   - Top_k基于固定数量的最可能选项进行选择（离散调节）
 
3. 一般用途：
   - 大多数情况下，只推荐temperature，需要平衡创意性和相关性的场景用高温度，高度可控和一致性输出的场景用低温度



默认值因模型而异,可以使用genai.get_model函数返回的Model对象的Model.temperature属性查看。

In [15]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-8b-exp-0827


In [16]:
genai.get_model("models/gemini-1.5-flash-exp-0827")

Model(name='models/gemini-1.5-flash-exp-0827',
      base_model_id='',
      version='exp-0827',
      display_name='Gemini 1.5 Flash Experimental 0827',
      description='Fast and versatile multimodal model for scaling across diverse tasks',
      input_token_limit=1048576,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [17]:
genai.get_model("models/gemini-1.5-pro-exp-0827")

Model(name='models/gemini-1.5-pro-exp-0827',
      base_model_id='',
      version='exp-0827',
      display_name='Gemini 1.5 Pro Experimental 0827',
      description='Mid-size multimodal model that supports up to 2 million tokens',
      input_token_limit=2097152,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [19]:
prompt = "讲解Python的类与实例，让初中生也能听懂"
# model = genai.GenerativeModel('models/gemini-1.5-flash-exp-0827',generation_config={
#     "temperature": 2,
# })
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 2,
})
print(response.text)

##  想象一下，制作披萨 

想象你想要制作一份披萨，你该怎么做呢？

* 首先，你需要准备一些 **基础材料**，比如面粉、水、酵母等。这些材料可以用来做各种不同口味的披萨。
* 然后，你需要选择 **披萨的类型**，例如你要做的是芝士披萨、海鲜披萨还是蔬菜披萨？
* 最后，你还要添加 **各种配料**，比如芝士、番茄酱、海鲜、蔬菜等，让披萨更有特色。

### 类：披萨的制作模板

在 Python 中，**类**就像披萨的制作模板，它定义了一系列基础材料和制作方法，可以用来制作不同的披萨。

* **基础材料**：对应类里的 **属性**，例如披萨的尺寸、形状、面皮种类等。
* **制作方法**：对应类里的 **方法**，例如添加配料、烤制披萨等。

### 实例：具体的一份披萨

**实例** 就是根据类模板创建的具体披萨，它包含了具体的基础材料和方法。

* 每个披萨都有自己的 **尺寸**、**形状**、**面皮种类** 等等，这些是具体的 **属性值**。
* 你可以选择添加不同的 **配料**，执行不同的 **烤制方法**，这些都是 **方法的应用**。

### 示例

假设我们有一个名为 "Pizza" 的类，它定义了披萨的基本信息：

```python
class Pizza:
  def __init__(self, size, shape, dough_type):
    self.size = size
    self.shape = shape
    self.dough_type = dough_type

  def add_topping(self, topping):
    print(f"Adding {topping} to the pizza!")
```

我们可以根据这个类，创建两个实例，分别代表不同的披萨：

```python
pizza1 = Pizza("Large", "Round", "Thin")
pizza2 = Pizza("Medium", "Square", "Thick")
```

现在我们有了两份具体的披萨：

* `pizza1` 是一个大尺寸、圆形、薄底的披萨
* `pizza2` 是一个中尺寸、方形、厚底的披萨

然后，我们可以使用 `add_topping` 方法为它们添

In [49]:
prompt = "讲解Python的类与实例，让初中生也能听懂"
# model = genai.GenerativeModel('models/gemini-1.5-flash-exp-0827',generation_config={
#     "temperature": 2,
# })
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 2,
})
print(response.text)

## Python的类与实例：造汽车的模具和汽车本身

想象一下，你想造一辆汽车。你不会每次都从零开始造，对吧？你肯定需要一个**模具**，这个模具决定了汽车的样子、零件等等。

在Python中，**类**就像汽车的模具，它定义了一种事物的**蓝图**。这个蓝图里包含了这个事物应该有哪些**特征**（比如汽车的颜色、速度）和可以执行的**动作**（比如汽车启动、加速）。

**实例**就像根据模具造出来的汽车本身。每一个实例都拥有自己的**具体特征**，比如一辆红色的跑车、一辆蓝色的卡车，它们都来自同一个模具（类），但又各有不同。


**举个例子：**

假设我们想用Python描述一个“狗”：

```python
class Dog:  # 定义一个叫做Dog的类，就像模具
    def __init__(self, name, color):  # 初始化方法，设置狗狗的初始特征
        self.name = name   # 狗狗的名字
        self.color = color # 狗狗的颜色
    
    def bark(self):  # 狗狗叫的方法，定义狗狗的动作
        print("汪汪汪！")
```

* `class Dog:`：告诉Python我们正在定义一个叫做Dog的类。
* `__init__`：这个方法叫做**构造方法**，它会在创建实例时自动执行，用来设置实例的初始特征。
* `self.name` 和 `self.color`：是狗狗的特征，每个狗狗都有自己的名字和颜色。
* `bark`：是一个方法，定义了狗狗的行为——叫。


现在，我们可以用这个“模具”来制造两只狗狗：

```python
dog1 = Dog("旺财", "黄色")  # 创建一个名叫“旺财”，颜色是黄色的狗狗实例
dog2 = Dog("小白", "白色")  # 创建一个名叫“小白”，颜色是白色的狗狗实例

dog1.bark() # 旺财叫
dog2.bark() # 小白叫 
```

这里，`dog1` 和 `dog2` 就是两个**实例**，它们都是Dog类创建出来的。它们分别拥有自己的名字和颜色，但是它们都能执行`bark`方法，也就是都能叫。


**总结一下：**

* **类**：定义了一

In [21]:
prompt = "讲解Python的类与实例，让初中生也能听懂"
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 0,
})
print(response.text)

##  Python 的类和实例：就像造汽车一样！

想象一下，你想造一辆汽车。你首先需要一个 **蓝图**，它描述了汽车的各个部分，比如车轮、发动机、车身等等。这个蓝图就是 **类**。

然后，根据这个蓝图，你可以造出很多 **具体的车**，比如红色的轿车、黑色的SUV等等。这些具体的车就是 **实例**。

**类**就像一个模具，它定义了所有汽车共有的属性和行为。**实例**则是根据这个模具制造出来的具体产品。

**举个例子：**

假设我们要造一辆汽车，它有以下属性：

* 颜色
* 品牌
* 型号

它也有以下行为：

* 启动
* 加速
* 刹车

我们可以用 Python 代码来定义这个汽车类：

```python
class Car:
  def __init__(self, color, brand, model):
    self.color = color
    self.brand = brand
    self.model = model

  def start(self):
    print("汽车启动了！")

  def accelerate(self):
    print("汽车加速了！")

  def brake(self):
    print("汽车刹车了！")
```

这段代码定义了一个名为 `Car` 的类。

* `__init__` 方法是类的构造函数，它用来初始化实例的属性。
* `start`、`accelerate` 和 `brake` 方法定义了汽车的行为。

现在，我们可以根据这个类创建多个汽车实例：

```python
my_car = Car("红色", "宝马", "3系")
your_car = Car("黑色", "奔驰", "C级")
```

`my_car` 和 `your_car` 都是 `Car` 类的实例，它们拥有相同的属性和行为，但属性值不同。

我们可以使用实例的属性和方法：

```python
print(my_car.color)  # 输出：红色
my_car.start()  # 输出：汽车启动了！
```

**总结：**

* 类就像一个蓝图，定义了对象的属性和行为。
* 实例是根据类创建的具体对象。
* 类可以用来创建多个实例，每个实例都有自己

In [48]:
model = genai.GenerativeModel('models/gemini-1.5-flash-exp-0827')

prompt = "讲解Python的类与实例，让初中生也能听懂"
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 0,
})
print(response.text)

## Python的类与实例：造汽车的模具和汽车本身

想象一下，你想造很多辆汽车，是不是可以先设计一个**汽车的模具**，然后根据这个模具来制造出各种各样的汽车呢？

在Python中，**类**就像这个汽车的模具，它定义了汽车应该有哪些部件（属性），以及汽车可以做什么（方法）。

**实例**就像根据模具造出来的汽车，它拥有类定义的所有部件，并且可以执行类定义的所有动作。

**举个例子：**

假设我们想用Python定义一个“汽车”类：

```python
class Car:
  def __init__(self, color, brand):
    self.color = color  # 汽车的颜色
    self.brand = brand  # 汽车的品牌
  
  def start(self):
    print("汽车启动了！")

  def stop(self):
    print("汽车停止了！") 
```

**解释：**

* `class Car:`  定义了一个名为“Car”的类，就像定义了一个汽车模具。
* `__init__(self, color, brand):`  是类的构造方法，就像模具的初始设置，它规定了每辆汽车都需要有颜色和品牌这两个属性。
* `self.color = color` 和 `self.brand = brand`  将传入的颜色和品牌信息存储到汽车实例中。
* `start(self)` 和 `stop(self)` 是汽车可以执行的动作，也就是方法。


现在，我们可以根据这个“Car”类来创建具体的汽车实例：

```python
my_car = Car("红色", "宝马")  # 创建一辆红色宝马汽车
your_car = Car("蓝色", "奔驰") # 创建一辆蓝色奔驰汽车
```

**解释：**

* `my_car` 和 `your_car` 就是根据“Car”类创建的两个汽车实例，就像两辆不同的汽车。
* `Car("红色", "宝马")`  表示使用“Car”类创建了一个实例，并设置了颜色为红色，品牌为宝马。


我们可以使用实例来访问属性和调用方法：

```python
print(my_car.color)  # 输出：红色
my_car.star

In [23]:
prompt = "讲解Python的类与实例，让初中生也能听懂"
response = model.generate_content(contents=prompt, generation_config={
    "top_k": 100,
})
print(response.text)

##  Python的类和实例：像盖房子一样造东西！

想象一下，你想盖一座房子。你不会直接拿砖块、水泥、木材就开始乱搭吧？你需要先设计一个**蓝图**，这个蓝图就相当于**类**，它描述了房子应该长什么样，有哪些房间，以及每个房间应该包含哪些东西。

**类**就好比盖房子的蓝图，它定义了事物的属性和行为。

**实例**就像根据蓝图盖出来的房子，每个房子都根据蓝图建造，但是每个房子都有自己的地址、颜色和内部装饰，这些就是**实例的属性**。房子还可以做一些事情，比如开灯、关门，这些就是**实例的行为**。

**举个例子：**

**类：**汽车
**属性：**颜色、品牌、型号、速度
**行为：**加速、刹车、转向

**实例：**一辆红色的宝马3系
**属性：**颜色是红色，品牌是宝马，型号是3系，速度是0
**行为：**可以加速、刹车、转向

**Python代码：**

```python
class Car:  # 定义汽车类
    def __init__(self, color, brand, model):  # 初始化方法，设置属性
        self.color = color
        self.brand = brand
        self.model = model
        self.speed = 0  # 初始速度为0

    def accelerate(self, speed_increase):  # 加速方法
        self.speed += speed_increase

    def brake(self):  # 刹车方法
        self.speed = 0

# 创建一个汽车实例
my_car = Car("红色", "宝马", "3系")

# 打印汽车的属性
print(my_car.color)  # 输出：红色
print(my_car.brand)  # 输出：宝马

# 让汽车加速
my_car.accelerate(50)

# 打印汽车的速度
print(my_car.speed)  # 输出：50

# 让汽车刹车
my_car.brake()

# 打印汽车的速度
print(my_car.speed)  # 输出：0
```

**总结：**



In [50]:
prompt = "创作一个关于小猫和秋天的四句儿歌"
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 0,
})
print(response.text)

小猫咪，毛茸茸，
秋风吹，落叶红。
抓抓叶，玩玩球，
秋天里，乐悠悠。 



In [51]:
prompt = "创作一个关于小猫和秋天的四句儿歌"
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 0,
})
print(response.text)

小猫咪，毛茸茸，
秋风吹，落叶红。
抓抓叶，玩玩球，
秋天里，乐悠悠。 



In [30]:
prompt = "创作一个关于小猫和秋天的四句儿歌"
response = model.generate_content(contents=prompt, generation_config={
    "temperature": 2,
})
print(response.text)

小猫穿毛衣， 
秋风吹得暖洋洋。 
落叶满地脆，
喵喵欢快玩耍忙。 


In [57]:
prompt = "创作一个关于小猫和秋天的四句儿歌"
response = model.generate_content(contents=prompt, generation_config={
    "top_k": 1
})
print(response.text)

小猫咪，毛茸茸，
秋风吹，落叶红。
抓抓叶，跳跳高，
快乐玩耍喵喵叫。 



stop_sequences参数

generation_config参数包括：
1. candidate_count: 生成的候选回答数量(目前只能为1)
2. max_output_tokens: 输出的最大token数
3. temperature: 控制输出随机性(0-1之间)
4. top_p和top_k: 控制词的选择范围  
5. stop_sequences: 指定停止生成的标记
6. response_mime_type: 指定输出格式
7. response_schema: 定义JSON输出结构


停止序列允许我们提供一组字符串（最多 5 个），当Gemini在生成响应时遇到这些字符串时，就会停止生成。这是一种告诉大模型 "如果你生成了这个序列，就停止生成其他内容"的方法。
如果指定，API 将在停止序列首次出现时停止。停止序列不会包含在响应中。

想一想，为什么大模型要创造这个参数呢？我们可以怎样的应用呢？

In [58]:
prompt = "一句话总结朝鲜和韩国的历史与关系,要准确详细"
response = model.generate_content(contents=prompt, generation_config={
    "stop_sequences": ["。","，"]
})
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\u671d\u9c9c\u548c\u97e9\u56fd\u7684\u5386\u53f2\u6e90\u4e8e\u53e4\u4ee3\u671d\u9c9c\u534a\u5c9b"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
           

In [59]:
response.text

'朝鲜和韩国的历史源于古代朝鲜半岛'

我要生成一堆语料，内容，并让其可以提取出来：

In [83]:
prompt = "列出三种水果，每种水果用<fruit>标签包围。"
response = model.generate_content(contents=prompt, generation_config={
    "stop_sequences": ["</fruit>"]
})
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "<fruit>\u82f9\u679c"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
      "usage_metadata": {
        "prompt_t

In [84]:
response.text

'<fruit>苹果'

In [19]:
prompt = "列出尽可能多的，几百种水果，每种水果用<fruit>标签包围。开始和结束用<content>,</content>包围"
response = model.generate_content(contents=prompt, generation_config={
    "stop_sequences": ["</content>"]
})
print(response.text)

<content>
<fruit>苹果</fruit>
<fruit>杏</fruit>
<fruit>鳄梨</fruit>
<fruit>香蕉</fruit>
<fruit>浆果</fruit>
<fruit>黑莓</fruit>
<fruit>甜菜根</fruit>
<fruit>比利时莓</fruit>
<fruit>黑加仑</fruit>
<fruit>樱桃</fruit>
<fruit>椰子</fruit>
<fruit>酸橙</fruit>
<fruit>柑橘</fruit>
<fruit>葡萄柚</fruit>
<fruit>覆盆子</fruit>
<fruit>醋栗</fruit>
<fruit>刺梨</fruit>
<fruit>日期</fruit>
<fruit>无花果</fruit>
<fruit>凤梨</fruit>
<fruit>葡萄</fruit>
<fruit>葡萄柚</fruit>
<fruit>西印度樱桃</fruit>
<fruit>哈密瓜</fruit>
<fruit>红莓</fruit>
<fruit>柠檬</fruit>
<fruit>酸橙</fruit>
<fruit>荔枝</fruit>
<fruit>龙眼</fruit>
<fruit>芒果</fruit>
<fruit>木瓜</fruit>
<fruit>蜜瓜</fruit>
<fruit>猕猴桃</fruit>
<fruit>橙子</fruit>
<fruit>桃子</fruit>
<fruit>梨</fruit>
<fruit>百香果</fruit>
<fruit>菠萝</fruit>
<fruit>石榴</fruit>
<fruit>李子</fruit>
<fruit>番石榴</fruit>
<fruit>山竹</fruit>
<fruit>草莓</fruit>
<fruit>树莓</fruit>
<fruit>覆盆子</fruit>
<fruit>西柚</fruit>
<fruit>杨梅</fruit>
<fruit>油桃</fruit>
<fruit>西瓜</fruit>
<fruit>冬瓜</fruit>
<fruit>柠檬草</fruit>
<fruit>罗望子</fruit>
<fruit>番木瓜</fruit>
<fruit>沙棘</fruit>
<

Json模式——结构化模型的输出

可以在提示中设置所需的结构化模型的输出，gemini也单独为模型提供了架构与参数。

数据提取和分析： 在这种场景中，结构化输出可以直接被用于数据分析或数据库插入。
内容生成： 在内容生成任务中，结构化输出可以帮助控制生成内容的格式和结构。例如，生成一篇带有特定结构的文章：

我们先看看**只在prompt提示中设置所需的结构化模型的输出**，刚才的stop sequnce中就有关于使用XML标签来控制结构化输出的案例，还有什么其他可能的运用呢？

In [ ]:
messages = [
    {"role": "user",
     "content": "写一篇关于气候变化的短文，包含以下部分：<introduction>引言</introduction><body>正文</body><conclusion>结论</conclusion>"}
]

In [ ]:
messages = [
    {"role": "user",
     "content": "提供一个5步计划来学习一门新语言。每个步骤都应包含<step>标题</step>和<description>描述</description>。"}
]

要求模型生成特定数量的问答对，并可以分隔问题和答案，

In [25]:
prompt = "请生成关于人工智能的问答对。使用以下格式：\n1. Q: [问题]\nA: [答案]\n2. Q: [问题]\nA: [答案]\n3. Q: [问题]\nA: [答案]"
response = model.generate_content(contents=prompt)
print(response.text)

1. Q: 什么是人工智能？
A: 人工智能 (AI) 是指计算机科学的一个分支，致力于构建能够执行通常需要人类智能的任务的智能代理。这些任务包括学习、推理、问题解决、感知、自然语言理解和决策等。

2. Q: 人工智能有哪些应用？
A: 人工智能的应用非常广泛，包括：
    * **自动驾驶汽车**:  使用AI进行导航和控制。
    * **医学诊断**: 利用AI分析医学图像和数据以协助诊断疾病。
    * **语音助手**: 例如Siri和Alexa，使用AI理解和回应人类语音指令。
    * **个性化推荐**:  电商平台和流媒体服务使用AI推荐商品和内容。
    * **欺诈检测**: 金融机构使用AI检测异常交易和防止欺诈。


3. Q: 人工智能的未来发展趋势是什么？
A: 人工智能的未来发展趋势包括：
    * **更强大的AI模型**:  随着计算能力的提高和算法的改进，AI模型将变得更加强大和智能。
    * **更广泛的应用**: AI将被应用于更多领域，例如教育、农业、制造业等。
    * **更加注重伦理和社会影响**: 随着AI的普及，人们将更加关注AI的伦理问题和社会影响，例如数据隐私、算法偏见和工作岗位替代等。 



In [26]:
import json

prompt = "生成3个关于人工智能的问答对，以JSON格式输出。格式如下：\n[{\"question\": \"问题1\", \"answer\": \"答案1\"}, {\"question\": \"问题2\", \"answer\": \"答案2\"}, {\"question\": \"问题3\", \"answer\": \"答案3\"}]"
response = model.generate_content(contents=prompt)
qa_pairs = json.loads(response.text)
for pair in qa_pairs:
    print(f"Q: {pair['question']}")
    print(f"A: {pair['answer']}\n")

Q: 什么是人工智能？
A: 人工智能（AI）是指计算机或机器展现出通常与人类智能相关的认知功能的能力，例如学习、解决问题和决策。

Q: 人工智能有哪些应用？
A: 人工智能应用广泛，包括自动驾驶汽车、语音助手、图像识别、医疗诊断、金融交易和机器人等领域。

Q: 人工智能的未来发展趋势是什么？
A: 人工智能的未来发展趋势包括更强大的计算能力、更先进的算法、更广泛的数据集以及更深入的伦理和社会影响研究。



In [27]:
from bs4 import BeautifulSoup

prompt = "生成3个关于人工智能的问答对，使用HTML标签格式。例如：<qa><q>问题</q><a>答案</a></qa>"
response = model.generate_content(contents=prompt)
soup = BeautifulSoup(response.text, 'html.parser')
qa_pairs = soup.find_all('qa')
for pair in qa_pairs:
    print(f"Q: {pair.q.text}")
    print(f"A: {pair.a.text}\n")

Q: 什么是人工智能?
A: 人工智能 (AI) 是指计算机或机器展现出类似人类智能的能力，例如学习、解决问题和决策。它涉及到开发能够执行通常需要人类智能的任务的计算机程序和算法。

Q: 人工智能有哪些应用?
A: 人工智能的应用非常广泛，包括：自动驾驶汽车、医学诊断、金融交易、语音识别、图像识别、自然语言处理、机器人技术等等。它正在改变着许多行业和我们的日常生活。

Q: 人工智能有哪些潜在风险?
A: 人工智能的潜在风险包括：工作岗位流失、算法偏见、隐私和安全问题、自主武器系统的伦理问题等等。了解和应对这些风险对于确保人工智能的负责任发展至关重要。



Google特有的Json的格式——response_mime_type，response_schema参数

最新的模型（1.5 及更高版本）开放的功能让我们可以直接传递结构化对象（或等效的 python 类型），并且输出将严格遵循该模式。

1. candidate_count: 生成的候选回答数量(目前只能为1)
2. max_output_tokens: 输出的最大token数
3. temperature: 控制输出随机性(0-1之间)
4. top_p和top_k: 控制词的选择范围  
5. stop_sequences: 指定停止生成的标记
6. response_mime_type: 指定输出格式
7. response_schema: 定义JSON输出结构

通过在generation_config参数中指定respose_mime_type来激活 JSON 模式：

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash-latest",
                              generation_config={"response_mime_type": "application/json"})

In [28]:

prompt = "生成3个关于人工智能的问答对"
response = model.generate_content(contents=prompt, generation_config={"response_mime_type": "application/json"})
print(response.text)

[{"question": "什么是人工智能？", "answer": "人工智能（AI）是指计算机或机器展现出类似人类智能的能力，例如学习、推理、解决问题和决策。"}, {"question": "人工智能有哪些应用？", "answer": "人工智能的应用非常广泛，包括自动驾驶、语音助手、图像识别、医疗诊断、金融预测等等。"}, {"question": "人工智能的未来发展趋势是什么？", "answer": "人工智能的未来发展趋势包括更强大的算法、更广泛的数据应用、更深入的行业融合以及更人性化的交互体验。"}]



当你在 generation_config 中指定 response_schema 时，你实际上是在为生成模型提供一个模板，描述了生成内容应该遵循的结构。

方式一：使用 TypedDict 和类型提示
当您使用 list[QAPair] 作为 response_schema 时，genai 库内部会检测到这是一个类型提示，并调用 _schema_for_class(cls) 方法将其转换为 JSON Schema 字典。
方式二：使用 JSON Schema 字典
这种方式直接提供了符合 JSON Schema 规范的字典，genai 直接使用该字典，无需额外的转换步骤。

建议选择 TypedDict。它简单明了，帮助我们理解数据结构和类型。


1. TypedDict 方式

定义方法：
```python
from typing import TypedDict

class Person(TypedDict):
    name: str
    age: int
```

优点：
- 提供静态类型检查：Python 的类型检查器（如 mypy）可以识别这种结构，帮助在开发阶段捕获类型错误。
- 代码更易读：结构清晰，直观地展示了数据的组成。

缺点：
- 对复杂数据结构支持较弱：难以表达嵌套结构、列表的具体内容类型等。

适用场景：适合初学者和需要清晰结构的项目。

原因：TypedDict 是 Python 的原生支持，学习成本低，对于简单的数据结构来说足够使用，同时能提供基本的类型安全。

2. JSON Schema 字典方式

定义方法：
```python
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "integer"}
    },
    "required": ["name", "age"]
}
```

优点：
- 灵活性高：可以定义复杂的嵌套结构、数组内容、条件验证等。
- 易于与外部系统集成：JSON Schema 是广泛使用的标准。

缺点：
- 需手动维护结构：容易出现拼写错误或结构错误。
- 缺乏静态类型检查：Python 无法直接检查字典的正确性。

适用场景：适合与外部API交互或需要复杂验证的项目。

原因：JSON Schema 提供了丰富的验证选项和灵活的结构定义能力，适合处理复杂的数据结构和验证需求。

3. protos.Schema 方式

定义方法：
```python
from google.ai.generativelanguage_v1beta.types import Schema, Type

schema = Schema(
    type_=Type.OBJECT,
    properties={
        "name": Schema(type_=Type.STRING),
        "age": Schema(type_=Type.INTEGER)
    }
)
```

优点：
- 与 genai 库深度集成：提供了完整的类型支持和验证。
- 强类型：可以捕获更多潜在错误。

缺点：
- 使用复杂：需要了解 protos 库的使用方法。
- 学习曲线陡峭：对于不熟悉 Protocol Buffers 的开发者来说可能较难上手。

适用场景：适合高级用户或需要严格类型支持的项目。

原因：protos.Schema 提供了最严格的类型检查和最完整的与 genai 库的集成，但也因此增加了使用的复杂性。

总结：
1. 对于简单项目或快速原型开发，TypedDict 是最佳选择。
2. 如果需要处理复杂的数据结构或与外部系统交互，JSON Schema 字典方式更合适。
3. 对于大型项目或需要严格类型控制的场景，protos.Schema 是最佳选择，尽管它有一定的学习曲线。


In [60]:
from typing_extensions import TypedDict


# 定义 TypedDict
class QAPair(TypedDict):
    question: str
    answer: str  # 使用小写字母

# 创建 GenerativeModel 实例
model = genai.GenerativeModel("gemini-1.5-pro-latest")

# 生成内容
result = model.generate_content(
    "列出几个关于LLM大模型应用开发的常见问题及其答案。",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema=list[QAPair]  # 或者使用 typing.List[QAPair]
    ),
)

print(result)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "[{\"answer\": \"\u9009\u62e9\u5408\u9002\u7684\u9884\u8bad\u7ec3\u6a21\u578b\u662fLLM\u5e94\u7528\u5f00\u53d1\u7684\u7b2c\u4e00\u6b65\uff0c\u9700\u8981\u6839\u636e\u5177\u4f53\u5e94\u7528\u573a\u666f\u7684\u9700\u6c42\uff0c\u9009\u62e9\u5177\u5907\u76f8\u5173\u77e5\u8bc6\u548c\u80fd\u529b\u7684\u6a21\u578b\uff0c\u5e76\u5173\u6ce8\u6a21\u578b\u7684\u89c4\u6a21\u3001\u6027\u80fd\u3001\u6210\u672c\u7b49\u56e0\u7d20\u3002\", \"question\": \"\u5982\u4f55\u9009\u62e9\u5408\u9002\u7684\u9884\u8bad\u7ec3\u6a21\u578b\uff1f\"}, {\"answer\": \"\u5fae\u8c03\u662f\u5c06\u9884\u8bad\u7ec3\u6a21\u578b\u8fc1\u79fb\u5230\u7279\u5b9a\u4efb\u52a1\u4e0a\u7684\u5173\u952e\u6b65\u9aa4\uff0c\u9700\u8981\u51c6\u5907\u9ad8\u8d28\u91cf\u7684\u6807\u6ce8\u6570\u636e\uff0c\u9009\u62e9\

In [61]:
print(result.text)

[{"answer": "选择合适的预训练模型是LLM应用开发的第一步，需要根据具体应用场景的需求，选择具备相关知识和能力的模型，并关注模型的规模、性能、成本等因素。", "question": "如何选择合适的预训练模型？"}, {"answer": "微调是将预训练模型迁移到特定任务上的关键步骤，需要准备高质量的标注数据，选择合适的微调方法和超参数，并进行充分的实验和评估，以获得最佳的模型性能。", "question": "如何进行模型微调？"}, {"answer": "高效的提示词工程可以引导LLM生成更准确、更符合预期的输出结果，可以通过设计清晰明确的任务指令、提供丰富的上下文信息、使用 few-shot learning 等技术来优化提示词。", "question": "如何设计有效的提示词？"}, {"answer": "评估LLM应用的性能需要定义明确的评估指标，例如准确率、召回率、F1分数等，并选择合适的评估数据集和评估方法，以客观、全面地衡量模型的性能表现。", "question": "如何评估LLM应用的性能？"}, {"answer": "部署LLM应用需要考虑计算资源、服务性能、安全性等因素，可以选择云端部署或本地部署的方式，并根据实际需求进行相应的优化和配置，以确保应用的稳定性和可靠性。", "question": "如何部署LLM应用？"}, {"answer": "模型偏差可能导致LLM应用生成不公平、不准确或有害的输出结果，需要采取措施识别和缓解模型偏差，例如使用去偏数据集、设计公平性约束、进行模型可解释性分析等。", "question": "如何解决模型偏差问题？"}, {"answer": "保护用户隐私是LLM应用开发的重要原则，需要遵守相关的数据隐私法规，并采取技术措施保护用户数据的安全和隐私，例如数据脱敏、差分隐私等。", "question": "如何保护用户隐私？"}] 


In [62]:
# 定义 JSON Schema
response_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "question": {"type": "string"},
            "answer": {"type": "string"}
        },
        "required": ["question", "answer"]
    }
}

# 创建 GenerativeModel 实例
model = genai.GenerativeModel("gemini-1.5-pro-latest")

# 生成内容
result = model.generate_content(
    "列出几个关于面试LLM大模型应用开发的常见问题及其答案。",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema=response_schema
    ),
)

print(result)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "[{\"answer\": \"LLM\u5927\u6a21\u578b\u5e94\u7528\u5f00\u53d1\u9700\u8981\u638c\u63e1\u54ea\u4e9b\u5173\u952e\u6280\u672f\uff1f\\\\n\\\\n- \u719f\u6089\u5e38\u89c1\u7684LLM\u6a21\u578b\uff0c\u4f8b\u5982GPT\u3001BERT\u7b49\uff0c\u4e86\u89e3\u5176\u4f18\u7f3a\u70b9\u548c\u9002\u7528\u573a\u666f\u3002\\\\n- \u638c\u63e1\u81ea\u7136\u8bed\u8a00\u5904\u7406\u7684\u57fa\u672c\u6982\u5ff5\u548c\u6280\u672f\uff0c\u4f8b\u5982\u5206\u8bcd\u3001\u8bcd\u5411\u91cf\u3001\u547d\u540d\u5b9e\u4f53\u8bc6\u522b\u7b49\u3002\\\\n- \u5177\u5907\u4e00\u5b9a\u7684\u7f16\u7a0b\u80fd\u529b\uff0c\u4f8b\u5982Python\uff0c\u719f\u6089\u5e38\u7528\u7684\u673a\u5668\u5b66\u4e60\u548c\u6df1\u5ea6\u5b66\u4e60\u6846\u67b6\uff0c\u4f8b\u5982TensorFlow\u3001PyTorch\u7b49\u3002\\\\n- \u4e86\u89e

In [63]:
print(result.text)

[{"answer": "LLM大模型应用开发需要掌握哪些关键技术？\\n\\n- 熟悉常见的LLM模型，例如GPT、BERT等，了解其优缺点和适用场景。\\n- 掌握自然语言处理的基本概念和技术，例如分词、词向量、命名实体识别等。\\n- 具备一定的编程能力，例如Python，熟悉常用的机器学习和深度学习框架，例如TensorFlow、PyTorch等。\\n- 了解如何对LLM模型进行微调，以适应特定的应用场景。\\n- 熟悉云计算平台，例如AWS、Azure、Google Cloud等，了解如何部署和管理LLM应用。", "question": "LLM大模型应用开发需要哪些技术储备？"}, {"answer": "LLM大模型应用开发有哪些常见的挑战？\\n\\n- 模型训练数据量大，训练时间长，需要大量的计算资源。\\n- 模型推理速度慢，对实时性要求高的应用场景挑战较大。\\n- 模型的可解释性差，难以理解模型的决策过程。\\n- 模型容易受到攻击，例如对抗样本攻击，存在安全风险。\\n- 模型的部署和维护成本较高。", "question": "开发基于LLM大模型的应用有哪些挑战？"}, {"answer": "如何评估一个LLM大模型应用的效果？\\n\\n-  根据具体的应用场景，选择合适的评估指标。例如，对于文本生成任务，可以使用BLEU、ROUGE等指标来评估生成文本的质量；对于问答系统，可以使用准确率、召回率等指标来评估系统的性能。\\n-  使用真实的数据集进行评估。\\n-  与其他方法进行比较，例如传统的机器学习方法。", "question": "如何评估一个LLM大模型应用？"}, {"answer": "LLM大模型应用开发有哪些常见的应用场景？\\n\\n-  **文本生成**: 例如文章写作、诗歌创作、代码生成、聊天机器人等。\\n-  **自然语言理解**: 例如文本分类、情感分析、问答系统、信息抽取等。\\n-  **机器翻译**: 例如将一种语言翻译成另一种语言。\\n-  **语音识别**: 例如将语音转换成文本。\\n-  **图像生成**: 例如根据文本描述生成图像。", "question": "LLM大模型有哪些应用场景？"}, {"answer": "如何处理LLM大模型应用开发中的数据安全和隐私问题？\\

In [ ]:
result = model.generate_content(
    ["What kind of instrument is this:", organ],
    generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum",
        response_schema={
            "type": "STRING",
            "enum": ["Percussion", "String", "Woodwind", "Brass", "Keyboard"],
        },
    ),
)
print(result)  # Keyboard

In [ ]:
import enum
from typing_extensions import TypedDict


class Grade(enum.Enum):
    A_PLUS = "a+"
    A = "a"
    B = "b"
    C = "c"
    D = "d"
    F = "f"


class Recipe(TypedDict):
    recipe_name: str
    grade: Grade


model = genai.GenerativeModel("gemini-1.5-pro-latest")

result = model.generate_content(
    "List about 10 cookie recipes, grade them based on popularity",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json", response_schema=list[Recipe]
    ),
)
print(result)  # [{"grade": "a+", "recipe_name": "Chocolate Chip Cookies"}, ...]

1. candidate_count: 生成的候选回答数量(目前只能为1)
2. max_output_tokens: 输出的最大token数
3. temperature: 控制输出随机性(0-1之间)
4. top_p和top_k: 控制词的选择范围  
5. stop_sequences: 指定停止生成的标记
6. response_mime_type: 指定输出格式
7. response_schema: 定义JSON输出结构

safty settings


允许开发者对AI生成内容进行细粒度的安全控制,以避免产生有害或不适当的内容。

如果内容被判定不安全:
   - 输入提示可能会被阻止
   - 生成的内容可能会被阻止并不返回

safty settings可以在创建 GenerativeModel 时设置,也可以在每次请求时传入

Gemini 模型支持 4 种安全类别:骚扰、仇恨言论、色情内容和危险内容。

这4个分类，相当于safty settings中的键：

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240924164432.png)

每个分类可以设定的阻止阈值，相当于safty settings中的值：

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240924163842.png)

默认的阻止阈值是BLOCK_MEDIUM_AND_ABOVE	


API支持使用枚举值、整数或字符串来指定这些安全设置。

有两种主要的安全反馈:
   - promptFeedback: 对输入提示的安全检查
   - 对生成内容的安全检查(在Candidate对象中)，API返回的GenerateContentResponse包含安全相关的反馈信息。
 


我们将编写一个被阻止的提示，查看原因，然后调整过滤器以解除阻止。

In [59]:
unsafe_prompt = "我觉得在披萨上放菠萝太奇怪了，写一篇文章狠狠地讽刺它，特别粗鲁特别不尊重的说"
response = model.generate_content(unsafe_prompt)

对于每个候选答案，需要先检查 response.candidates.finish_reason 。

生成的候选回复的安全反馈在 Candidate.finishReason 和 Candidate.safetyRatings 中。


如果 candidate.finish_reason 是 FinishReason.STOP ，这意味着您的生成请求成功运行

如果 candidate.finish_reason 是 FinishReason.SAFETY ，这意味着您的生成请求因安全原因被阻止。这也意味着 response.text 结构将为空。

In [53]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "finish_reason": "SAFETY",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "MEDIUM"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 28,
        "total_token_count": 28
      }
    }),
)


In [54]:
print(response.candidates[0].finish_reason)

FinishReason.SAFETY


In [55]:
print(response.candidates[0].safety_ratings)

[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: MEDIUM
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
]


In [56]:
try:
    print(response.text)
except:
    print("No information generated by the model.")

No information generated by the model.


我们可以选择设置为不阻止内容。当然对于某些提示，Gemini 仍然将避免生成结果，即使我们将所有safty settings设置为无。

In [60]:
response = model.generate_content(
    unsafe_prompt,
    safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
        'SEXUAL': 'BLOCK_NONE',
        'DANGEROUS': 'BLOCK_NONE'
    })

In [61]:
try:
    print(response.text)
except:
    print("No information generated by the model.")

## 菠萝披萨：一种对人类文明的侮辱

各位爱好披萨的朋友们，你们是否也和我一样，对一种罪恶的邪恶食物感到深深的厌恶？没错，我说的就是披萨上的那颗“毒瘤”—— **菠萝**！ 

有些人可能还陶醉于“酸甜搭配”的幻觉，但他们忘记了：披萨是神圣的，是意大利传统美食的精华，是无数人心中最美好的味觉记忆！而菠萝，这颗生长在热带丛林的“野蛮水果”，**根本没有资格玷污这神圣的美味**！

想象一下，当你在享受着香气扑鼻的披萨时，突然咬到一块**冰冷甜腻的菠萝**，那感觉就像是在天堂吃了一口**猪食**！这酸涩的口感，这奇特的搭配，简直是在挑战你的味觉极限！ 

更可恶的是，一些披萨店为了迎合一些“奇奇怪怪”的口味，竟然把菠萝作为一种**标准配料**，这简直是**对传统披萨的亵渎**！难道他们不知道， **披萨只有经典的美味才配得上它的名字**吗？

所以，朋友们，让我们一起抵制菠萝披萨，**守护披萨的尊严！** 我们应该用更纯粹的食材，更经典的搭配，来展现披萨的魅力，而不是用这些**毫无道理的水果**来破坏它！

记住，披萨是神圣的，它**不需要**任何额外的装饰，更不需要**菠萝这颗“毒瘤”**！  


![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240924175116.png)

为了防止一些例外，我们自己调用API时，可以将safety_setting从默认值更改为最开放的方法。并放在初始化model处，而不是单次问答中。

这时候可能有小伙伴发现了两次用的并不一样。这是因为safety settings中具体的键值可以使用枚举值、整数或字符串来指定，这些在后端有工程化的转换。

In [ ]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE",
    },
]

In [39]:
model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

啊，这，这是什么！不用担心，这些都是内置的枚举类型，我们可以跳转查看。

In [70]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

safety_settings = {
    'hate': "low",
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    'DANGEROUS': 'BLOCK_NONE',
    "sexually_explicit": 1
}
model = genai.GenerativeModel(
    model_name="gemini-pro",
    safety_settings=safety_settings,
)
model

genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_LOW_AND_ABOVE: 1>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_LOW_AND_ABOVE: 1>, <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_LOW_AND_ABOVE: 1>},
    tools=None,
    system_instruction=None,
    cached_content=None
)

安全设置主要包括:
   - HarmCategory: 危害类别,如骚扰、仇恨言论等
   - HarmBlockThreshold: 阻止阈值
   - SafetyRating: 包含危害类别和概率的评分

In [66]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Please provide me with the image or a description of what you're referring to! I need more context to determine if something looks store-bought or homemade. \n\nFor example, you could say:\n\n* \"These cookies look very uniform and have perfect edges. Do these look store-bought or homemade?\"\n* \"This cake has a rustic, uneven frosting. Do these look store-bought or homemade?\"\n\nOnce you give me more information, I can help you assess the appearance and give my opinion!"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }

加一个子agent

In [ ]:
def make_gemini_client():
    generation_config = {
        "temperature": 0.7,
        "top_p": 1,
        "top_k": 1,
        "max_output_tokens": 2048,
    }

    safety_settings = [
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE",
        },
    ]

    model = genai.GenerativeModel(
        model_name="gemini-pro",
        generation_config=generation_config,
        safety_settings=safety_settings,
    )
    client = model.start_chat()
    return client


def pro_prompt_by_gemini(prompt: str, client) -> str:
    # TODO fix the type hint
    prompt = f"revise `{prompt}` to a DALL-E prompt, return the content in English only return the scene and detail"
    client.send_message(prompt)
    return client.last.text


gemini_client = make_gemini_client()